In [1]:
import os
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.transforms import RandomNodeSplit, RandomLinkSplit
from torch_geometric.loader import DataLoader, NeighborLoader

/home/wuxinchao/software/miniconda3/envs/pytorch-gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Plotting settings

colors = ["#3B7EA1", "#FDB515", "#D9661F", "#859438", "#EE1F60", "#00A598"]
sns.set(context="notebook", font_scale=1.3, style="ticks")
sns.set_palette(sns.color_palette(colors))
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['savefig.transparent'] = True
sc.settings._vector_friendly = True
DPI = 300
# GPU settings

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:

# data_path
save_path = "/home/wuxinchao/data/st_cite_data/totalVI_reproducibility/data/"
mdata = mu.read_h5mu(save_path + "/SNL_111.h5mu")
rna = mdata.mod["rna"]
protein = mdata.mod["prot"]

# Preprocess

In [8]:
# RNA
sc.pp.normalize_total(rna)
sc.pp.log1p(rna)
rna.obs_names_make_unique()

In [9]:
from muon import prot as pt

pt.pp.clr(protein)
mdata.update()

# Data input

## Spatial Data

In [4]:
data_path = "/home/wuxinchao/data/st_cite_data/"
prot_data_path = "B01825A4_protein_filter.csv"
rna_data_path = "B01825A4_rna_raw.csv"

In [ ]:
from utils import construct_spatial_adata

sp_mudata = construct_spatial_adata(
    data_path, 
    rna_data=rna_data_path, 
    prot_data=prot_data_path
)
sp_mudata

In [ ]:
rna_adata = sp_mudata.mod["rna"]
prot_adata = sp_mudata.mod["prot"]

In [10]:
sc.pp.normalize_total(rna_adata)
sc.pp.log1p(rna_adata)
rna_adata.obs_names_make_unique()
rna_adata.var['mt'] = rna_adata.var_names.str.startswith('MT-')
rna_adata.layers["counts"] = rna_adata.X.copy()
sc.pp.highly_variable_genes(
    rna_adata,
    n_top_genes=2000,
    flavor="seurat_v3",
    layer="counts",
)

sc.pp.scale(rna_adata, max_value=10)
sc.tl.pca(rna_adata, svd_solver="arpack")
# sc.pl.pca_variance_ratio(rna_adata, log=True)
sc.pp.neighbors(rna_adata, n_pcs=50)
sc.tl.umap(rna_adata)

/home/wuxinchao/software/miniconda3/envs/pytorch-gpu/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:197: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))
/home/wuxinchao/software/miniconda3/envs/pytorch-gpu/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [11]:
from muon import prot as pt

pt.pp.clr(prot_adata)
sp_mudata.update()

In [12]:
sp_mudata.write_h5mu(data_path + "sp_mudata.h5mu")

In [5]:
sp_mudata = mu.read_h5mu(data_path + "sp_mudata.h5mu")
sp_mudata

MuData object with n_obs × n_vars = 8628 × 25828
  obsm:	'spatial'
  2 modalities
    rna:	8628 x 25818
      var:	'mt', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std'
      uns:	'hvg', 'log1p', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap', 'spatial'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'connectivities', 'distances'
    prot:	8628 x 10
      obsm:	'spatial'

In [6]:
rna_adata = sp_mudata.mod["rna"]
prot_adata = sp_mudata.mod["prot"]

# Training with model

In [7]:
concat_data = np.concatenate(
    [rna_adata.X, prot_adata.X], axis=1
)

concat_spatial_encoding_data = np.concatenate(
    [rna_adata.X, prot_adata.X, sp_mudata.obsm["spatial"]], 
    axis=1
)

adj_mtx = rna_adata.obsp['connectivities'].toarray()
edge_index = adj_mtx.nonzero()
edge_index = torch.tensor(edge_index, 
        dtype=torch.long).contiguous()

scCITEseq_data = Data(x=torch.tensor(
    concat_data, dtype=torch.float), 
    edge_index=edge_index)

spCITEseq_data = Data(x=torch.tensor(
    concat_spatial_encoding_data, dtype=torch.float), 
    edge_index=edge_index)

/tmp/ipykernel_1439555/1871230178.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  edge_index = torch.tensor(edge_index,


In [8]:
num_splits = 2
num_val = 0.2
num_test = 0.2

tsf = RandomNodeSplit(num_splits=num_splits, 
                      num_val=num_val, 
                      num_test=num_test, 
                      key=None
                      )
training_data = tsf(scCITEseq_data)
spatial_training_data = tsf(spCITEseq_data)

In [12]:
# setting for the model
rna_input_dim = rna_adata.shape[1]
prot_input_dim = prot_adata.shape[1]
hidden_dim = 32
embedding_dim = 32
heads = 4
num_blocks = 2
permute = True
preserve_rate = 0.2
alpha = 0.4
beta = 0.4

# setting for the trainer
batch_size = 256
epochs = 25
mask_ratio = 0.85
num_splits = 2
device = torch.device(
    'cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
from train import Trainer

In [15]:
model_choice = "Graph Cross Attention"

trainer = Trainer(
    training_data, 
    model_choice=model_choice, 
    rna_input_dim=rna_input_dim, 
    prot_input_dim=prot_input_dim,
    hidden_dim=hidden_dim,
    embedding_dim=embedding_dim,
    heads=heads,
    num_blocks=num_blocks, 
    batch_size=batch_size,
    epochs=epochs,
    mask_ratio=mask_ratio,
    permute=permute,
    preserve_rate=preserve_rate,
    num_splits=num_splits,
    device=device,
    alpha=alpha,
    beta=beta,
    )

train_losses, val_losses = trainer.train()

Epoch 1/25 train_loss: 0.00420 val_loss: 0.00053
Epoch 2/25 train_loss: 0.00030 val_loss: 0.00024
Epoch 3/25 train_loss: 0.00014 val_loss: 0.00020
Epoch 4/25 train_loss: 0.00010 val_loss: 0.00020
Epoch 5/25 train_loss: 0.00009 val_loss: 0.00020
Epoch 6/25 train_loss: 0.00007 val_loss: 0.00019
Epoch 7/25 train_loss: 0.00007 val_loss: 0.00019
Epoch 8/25 train_loss: 0.00006 val_loss: 0.00019
Epoch 9/25 train_loss: 0.00005 val_loss: 0.00020
Epoch 10/25 train_loss: 0.00004 val_loss: 0.00019
Epoch 11/25 train_loss: 0.00004 val_loss: 0.00019
Epoch 12/25 train_loss: 0.00003 val_loss: 0.00018
Epoch 13/25 train_loss: 0.00003 val_loss: 0.00019
Epoch 14/25 train_loss: 0.00003 val_loss: 0.00018
Epoch 15/25 train_loss: 0.00002 val_loss: 0.00018
Epoch 16/25 train_loss: 0.00002 val_loss: 0.00019
Epoch 17/25 train_loss: 0.00002 val_loss: 0.00019
Epoch 18/25 train_loss: 0.00002 val_loss: 0.00019
Epoch 19/25 train_loss: 0.00002 val_loss: 0.00019
Epoch 20/25 train_loss: 0.00001 val_loss: 0.00019
Epoch 21/

In [16]:
model_choice = "Spatial Graph Cross Attention"

spatial_trainer = Trainer(
    spatial_training_data, 
    model_choice=model_choice, 
    rna_input_dim=rna_input_dim, 
    prot_input_dim=prot_input_dim,
    hidden_dim=hidden_dim,
    embedding_dim=embedding_dim,
    heads=heads,
    num_blocks=num_blocks, 
    batch_size=batch_size,
    epochs=epochs,
    mask_ratio=mask_ratio,
    permute=permute,
    preserve_rate=preserve_rate,
    num_splits=num_splits,
    device=device,
    alpha=alpha,
    beta=beta,
    )

spatial_train_losses, spatial_val_losses = spatial_trainer.train()

Epoch 1/25 train_loss: nan val_loss: nan
Epoch 2/25 train_loss: nan val_loss: nan
Epoch 3/25 train_loss: nan val_loss: nan
Epoch 4/25 train_loss: nan val_loss: nan
Epoch 5/25 train_loss: nan val_loss: nan
Epoch 6/25 train_loss: nan val_loss: nan
Epoch 7/25 train_loss: nan val_loss: nan
Epoch 8/25 train_loss: nan val_loss: nan
Epoch 9/25 train_loss: nan val_loss: nan
Epoch 10/25 train_loss: nan val_loss: nan
Epoch 11/25 train_loss: nan val_loss: nan
Epoch 12/25 train_loss: nan val_loss: nan
Epoch 13/25 train_loss: nan val_loss: nan
Epoch 14/25 train_loss: nan val_loss: nan
Epoch 15/25 train_loss: nan val_loss: nan
Epoch 16/25 train_loss: nan val_loss: nan
Epoch 17/25 train_loss: nan val_loss: nan
Epoch 18/25 train_loss: nan val_loss: nan
Epoch 19/25 train_loss: nan val_loss: nan
Epoch 20/25 train_loss: nan val_loss: nan
Epoch 21/25 train_loss: nan val_loss: nan
Epoch 22/25 train_loss: nan val_loss: nan
Epoch 23/25 train_loss: nan val_loss: nan
Epoch 24/25 train_loss: nan val_loss: nan
E